In [1]:
# Essential Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from PIL import Image

# Load Dataset Metadata
df_train = pd.read_csv('/kaggle/input/google-landmark-v2/train_landmark.csv')
df_test = pd.read_csv('/kaggle/input/google-landmark-v2/test_landmark.csv')

# Inspect Data
print("The Training Data")
print("=" * 100)
print(df_train.head(5))
print("=" * 100)

print("The Testing Data")
print("=" * 100)
print(df_test.head(5))
print("=" * 100)


The Training Data
                 id                                                url  \
0  6e158a47eb2ca3f6  https://upload.wikimedia.org/wikipedia/commons...   
1  202cd79556f30760  http://upload.wikimedia.org/wikipedia/commons/...   
2  3ad87684c99c06e1  http://upload.wikimedia.org/wikipedia/commons/...   
3  e7f70e9c61e66af3  https://upload.wikimedia.org/wikipedia/commons...   
4  4072182eddd0100e  https://upload.wikimedia.org/wikipedia/commons...   

   landmark_id  
0       142820  
1       104169  
2        37914  
3       102140  
4         2474  
The Testing Data
                 id
0  00016575233bc956
1  0001aadbcd8cb923
2  0002c06b2440a5f9
3  0002eb1ee5a5a6b2
4  000594dad986513e


In [2]:
df_train.shape

(4132914, 3)

In [ ]:
import requests
from tqdm import tqdm
import os
import shutil

# Create Directory for Images
# shutil.rmtree('/kaggle/working/train/', ignore_errors=True)
os.makedirs('/kaggle/working/train/', exist_ok=True)

def download_images(dataframe, save_dir, max_images=100000):
    os.makedirs(save_dir, exist_ok=True)
    downloaded = 0

    for idx, row in tqdm(dataframe.iterrows(), total=min(len(dataframe), max_images), desc='Downloading Images'):
        if downloaded >= max_images:
            print("Reached maximum download limit.")
            break

        img_id = row['id']
        img_url = row['url']
        img_path = os.path.join(save_dir, f"{img_id}.jpg")

        if os.path.exists(img_path):
            continue  # Skip already downloaded images

        try:
            response = requests.get(img_url, timeout=5)
            if response.status_code == 200:
                with open(img_path, 'wb') as f:
                    f.write(response.content)
                downloaded += 1
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {img_url}: {e}")
        
# Download up to 100,000 images
download_images(df_train, '/kaggle/working/train/', max_images=100000)
print("Number of images downloaded:", len(os.listdir('/kaggle/working/train/')))


In [ ]:
# Preprocess Dataset
train_df, val_df = train_test_split(df_train, test_size=0.2, random_state=42)

# Custom Dataset Class
class LandmarkDataset(Dataset):
    def __init__(self, dataframe, directory, transform=None, is_test=False):
        self.dataframe = dataframe
        self.directory = directory
        self.transform = transform
        self.is_test = is_test

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_id = self.dataframe.iloc[idx]['id']
        img_path = f'{self.directory}/{img_id}.jpg'
        try:
            image = Image.open(img_path).convert('RGB')
        except FileNotFoundError:
            print(f"Warning: {img_path} not found.")
            return None, None  # Return placeholders if image is missing

        if self.transform:
            image = self.transform(image)
        
        if self.is_test:
            return image, img_id
        
        label = self.dataframe.iloc[idx]['landmark_id']
        return image, label

# Image Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Dataset Instances
train_dataset = LandmarkDataset(train_df, '/kaggle/working/train', transform=transform)
val_dataset = LandmarkDataset(val_df, '/kaggle/working/train', transform=transform)
test_dataset = LandmarkDataset(df_test, '/kaggle/working/train', transform=transform, is_test=True)

# DataLoader Instances
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
# Using Pre-Trained Model for transfer Learning on the Dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"The device in use: {device}")

# Model Initialization
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(train_df['landmark_id'].unique()))
model = model.to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

In [ ]:
train_loader

In [ ]:
# Training the Model
def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, epochs=30):
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct = 0, 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()
        
        val_loss, val_correct = 0, 0
        model.eval()
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_correct += (outputs.argmax(1) == labels).sum().item()
        
        scheduler.step(val_loss)
        
        history['train_loss'].append(train_loss / len(train_loader))
        history['val_loss'].append(val_loss / len(val_loader))
        history['train_acc'].append(train_correct / len(train_loader.dataset))
        history['val_acc'].append(val_correct / len(val_loader.dataset))
        
        print(f'Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
    
    # Plot Training History
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # return history

# history = 
train_model(model, criterion, optimizer, scheduler, train_loader, val_loader)
# torch.save(model.state_dict())

In [ ]:
# Evaluateing the Model Performance
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in tqdm(val_loader, desc='Evaluation Loop'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = outputs.argmax(1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'Validation F1-Score: {f1}')

In [ ]:
# Test Predictions
correct, wrong = 0, 0
for images, img_ids in tqdm(test_loader, desc='Test Predictions'):
    images = images.to(device)
    outputs = model(images)
    preds = outputs.argmax(1)
    for i, img_id in enumerate(img_ids):
        predicted_class = preds[i].item()
        correct += 1  # Placeholder for actual comparison logic

# Pie Chart
labels = ['Correctly Classified', 'Wrongly Classified']
sizes = [correct, len(test_loader.dataset) - correct]
colors = ['#66b3ff', '#ff6666']
plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140)
plt.title('Test Dataset Classification Results')
plt.show()